# <center><font color = 'blue'> Sched Square

The aim of the square example is to place a set of small squares of
different sizes into a large square.

## Import lib

In [ ]:
try:
    import docplex.cp
except:
    !pip install docplex

In [ ]:
from docplex.cp.model import *

## Data

Size of the englobing square

In [ ]:
SIZE_SQUARE = 112

Sizes of the sub-squares

In [ ]:
SIZE_SUBSQUARE = [50, 42, 37, 35, 33, 29, 27, 25, 24, 19, 18, 17, 16, 15, 11, 9, 8, 7, 6, 4, 2]

## Modeling

### Create CPO model

In [ ]:
mdl = CpoModel()

### Create array of variables for sub-squares

In [ ]:
x = []
y = []
rx = pulse(0, 0, 0)
ry = pulse(0, 0, 0)

for i in range(len(SIZE_SUBSQUARE)):
    sq = SIZE_SUBSQUARE[i]
    vx = interval_var(size=sq, name="X" + str(i))
    vx.set_end((0, SIZE_SQUARE))
    x.append(vx)
    rx += pulse(vx, sq)

    vy = interval_var(size=sq, name="Y" + str(i))
    vy.set_end((0, SIZE_SQUARE))
    y.append(vy)
    ry += pulse(vy, sq)

### Create dependencies between variables

In [ ]:
for i in range(len(SIZE_SUBSQUARE)):
    for j in range(i):
        mdl.add((end_of(x[i]) <= start_of(x[j]))
                | (end_of(x[j]) <= start_of(x[i]))
                | (end_of(y[i]) <= start_of(y[j]))
                | (end_of(y[j]) <= start_of(y[i])))

### Set other constraints

In [ ]:
mdl.add(always_in(rx, 0, SIZE_SQUARE, SIZE_SQUARE, SIZE_SQUARE))
mdl.add(always_in(ry, 0, SIZE_SQUARE, SIZE_SQUARE, SIZE_SQUARE))

## Solve the model

### Define search phases

In [ ]:
mdl.set_search_phases([search_phase(x), search_phase(y)])

### Set your DOcloud credentials:
0. A first option is to set the DOcloud url and key directly in the model source file *(see below)*
1. For a persistent setting, create a Python file __docloud_config.py__ somewhere that is visible from the __PYTHONPATH__

In [ ]:
msol = mdl.solve(url="https://api-oaas.docloud.ibmcloud.com/job_manager/rest/v1/", 
                 key="ENTER YOUR KEY HERE",
                 TimeLimit=20,
                 LogPeriod=50000)

## Display Solution

### Print Solution

In [ ]:
print("Solution: ")
msol.print_solution()

### Import graphical tools

In [ ]:
import _utils_visu as visu
import matplotlib.pyplot as plt

*You can set __POP\_UP\_GRAPHIC=True__ if you prefer a pop up graphic window instead of an inline one.*

In [ ]:
POP_UP_GRAPHIC=False

In [ ]:
if msol and visu.is_visu_enabled():
    import matplotlib.cm as cm
    from matplotlib.patches import Polygon
    
    if not POP_UP_GRAPHIC:
        %matplotlib inline
    
    # Plot external square
    print("Plotting squares....")
    fig, ax = plt.subplots()
    plt.plot((0, 0), (0, SIZE_SQUARE), (SIZE_SQUARE, SIZE_SQUARE), (SIZE_SQUARE, 0))
    for i in range(len(SIZE_SUBSQUARE)):
        # Display square i
        (sx, sy) = (msol.get_var_solution(x[i]), msol.get_var_solution(y[i]))
        (sx1, sx2, sy1, sy2) = (sx.get_start(), sx.get_end(), sy.get_start(), sy.get_end())
        poly = Polygon([(sx1, sy1), (sx1, sy2), (sx2, sy2), (sx2, sy1)], fc=cm.Set2(float(i) / len(SIZE_SUBSQUARE)))
        ax.add_patch(poly)
        # Display identifier of square i at its center
        ax.text(float(sx1 + sx2) / 2, float(sy1 + sy2) / 2, str(SIZE_SUBSQUARE[i]), ha='center', va='center')
    plt.margins(0)
    plt.show()
